In [7]:
# GUARDAR OS DADOS GEOGRÁFICOS EM TIFF

In [8]:
from pathlib import Path
import os
import rasterio
import re
import numpy as np
import geopandas as gpd
from rasterio.enums import Resampling
from rasterio import features
from rasterio.warp import calculate_default_transform, reproject, Resampling
from glob import glob
import shutil
from pathlib import Path

In [9]:
aspect_path = r"../../Data/Raw/GIS_data/aspect/aspect/50m/copernicus/v2/aspect.asc"
elevation_path = r"../../Data/Raw/GIS_data/elevation/elevation/50m/copernicus/v2/elevation.asc"

last_fire_paths = list(Path(r"../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire").rglob("*.asc"))

fuel_paths = list(Path(r"../../Data/Raw/GIS_data/fuel_models/100m/portugal/PT-FireSprd").rglob("*.asc"))

GIS_paths = [aspect_path, elevation_path] + last_fire_paths + fuel_paths

print(GIS_paths)

['../../Data/Raw/GIS_data/aspect/aspect/50m/copernicus/v2/aspect.asc', '../../Data/Raw/GIS_data/elevation/elevation/50m/copernicus/v2/elevation.asc', WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2016/years_since_fire_2016.asc'), WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2017/years_since_fire_2017.asc'), WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2018/years_since_fire_2018.asc'), WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2019/years_since_fire_2019.asc'), WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2020/years_since_fire_2020.asc'), WindowsPath('../../Data/Raw/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/v2021/years_since_fire_2021.asc'), WindowsPath('../../Data/Raw/GIS_data/years_sin

In [10]:
for path in GIS_paths:
    path = Path(path)

    print(path)

    # substituir "Raw" por "Interim" no caminho
    tif_path = Path(str(path).replace(r"Raw", r"Interim")).with_suffix(".tif")

    print(tif_path)

    # garantir que a pasta existe
    os.makedirs(tif_path.parent, exist_ok=True)

    # só converte se ainda não existir
    if not tif_path.exists():
        with rasterio.open(path) as src:
            profile = src.profile
            profile.update(driver="GTiff")  # saída em GeoTIFF
            with rasterio.open(tif_path, "w", **profile) as dst:
                dst.write(src.read())
        print(f"✅ Convertido: {tif_path}")
    else:
        print(f"ℹ️ Já existe: {tif_path}")


..\..\Data\Raw\GIS_data\aspect\aspect\50m\copernicus\v2\aspect.asc
..\..\Data\Interim\GIS_data\aspect\aspect\50m\copernicus\v2\aspect.tif
ℹ️ Já existe: ..\..\Data\Interim\GIS_data\aspect\aspect\50m\copernicus\v2\aspect.tif
..\..\Data\Raw\GIS_data\elevation\elevation\50m\copernicus\v2\elevation.asc
..\..\Data\Interim\GIS_data\elevation\elevation\50m\copernicus\v2\elevation.tif
ℹ️ Já existe: ..\..\Data\Interim\GIS_data\elevation\elevation\50m\copernicus\v2\elevation.tif
..\..\Data\Raw\GIS_data\years_since_last_fire\years_since_last_fire\100m\portugal\isa\v2016\years_since_fire_2016.asc
..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\100m\portugal\isa\v2016\years_since_fire_2016.tif
ℹ️ Já existe: ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\100m\portugal\isa\v2016\years_since_fire_2016.tif
..\..\Data\Raw\GIS_data\years_since_last_fire\years_since_last_fire\100m\portugal\isa\v2017\years_since_fire_2017.asc
..\..\Data\Interim\GIS_data\years

In [11]:
# Caminhos
base_50m = Path("../../Data/Interim/GIS_data/years_since_last_fire/years_since_last_fire/50m/portugal/isa/")
base_100m = Path("../../Data/Interim/GIS_data/years_since_last_fire/years_since_last_fire/100m/portugal/isa/")
dest = Path("../../Data/Interim/GIS_data/years_since_last_fire/PT-FireSprd")

# Criar diretório destino se não existir
dest.mkdir(parents=True, exist_ok=True)

# Anos alvo
anos = [str(a) for a in range(2015, 2026)]

# Expressão regular para extrair ano
re_ano = re.compile(r"(20\d{2})")

def listar_tiffs(base_path):
    resultados = {}
    for root, _, files in os.walk(base_path):
        for f in files:
            if f.lower().endswith(".tif"):
                match = re_ano.search(f)
                if match:
                    ano = match.group(1)
                    if ano in anos and ano not in resultados:
                        resultados[ano] = Path(root) / f
    return resultados

# Recolher ficheiros
tiffs_50m = listar_tiffs(base_50m)
tiffs_100m = listar_tiffs(base_100m)

# Combinar com prioridade 50m
selecionados = {}
for ano in anos:
    if ano in tiffs_50m:
        selecionados[ano] = tiffs_50m[ano]
    elif ano in tiffs_100m:
        selecionados[ano] = tiffs_100m[ano]

# Copiar ficheiros
for ano, src in selecionados.items():
    destino = dest / src.name
    shutil.copy2(src, destino)
    print(f"✅ Copiado {src} → {destino}")

print("\n🎯 Concluído! Tiffs copiados para:", dest)


✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2015\years_since_fire_2015.tif → ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2015.tif


✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2016\years_since_fire_2016.tif → ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2016.tif
✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2017\years_since_fire_2017.tif → ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2017.tif
✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2018\years_since_fire_2018.tif → ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2018.tif
✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2019\years_since_fire_2019.tif → ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2019.tif
✅ Copiado ..\..\Data\Interim\GIS_data\years_since_last_fire\years_since_last_fire\50m\portugal\isa\v2020\years_since

In [12]:
# Caminhos
tiff_dir = os.path.normpath("../../Data/Interim/GIS_data/years_since_last_fire/PT-FireSprd")
COS_2018_PATH = os.path.normpath("../../Data/Raw/GIS_data/COS/COS2018v3-S2-shp/COS2018v3-S2.shp")
COS_2023_PATH = os.path.normpath("../../Data/Raw/GIS_data/COS/COS2023v1-S2-shp/COS2023v1-S2.shp")

TARGET_CRS = "EPSG:32629"

# Procurar todos os TIFFs recursivamente
tiff_files = [f for f in glob(os.path.join(tiff_dir, "**", "*.tif"), recursive=True) if "_p" not in f]

# Preparar COS: ler, filtrar e reprojetar para TARGET_CRS
def preparar_cos(path):
    print(f"\n📖 A carregar {os.path.basename(path)} ...")
    gdf = gpd.read_file(path)

    # Encontrar campo COS
    field = next((c for c in gdf.columns if c.lower().startswith("cos")), None)

    # Filtrar classes que começam por 3, 4 ou 5 (vegetação)
    gdf = gdf[gdf[field].astype(str).str.match(r"^[345]")]
    print(f"✅ {len(gdf)} polígonos de vegetação filtrados.")

    # Reprojetar para TARGET_CRS se necessário
    if gdf.crs != TARGET_CRS:
        print(f"🔄 Reprojetando COS → {TARGET_CRS}")
        gdf = gdf.to_crs(TARGET_CRS)

    return gdf

# Ler e preparar COS apenas uma vez
cos_2018 = preparar_cos(COS_2018_PATH)
cos_2023 = preparar_cos(COS_2023_PATH)

# Função para reprojetar raster para TARGET_CRS se necessário
def reproject_to_target(in_path, out_path):
    with rasterio.open(in_path) as src:
        if src.crs == TARGET_CRS:
            return in_path  # já está no CRS certo
        print(f"🔄 Reprojetando {os.path.basename(in_path)} → {TARGET_CRS}")
        transform, width, height = calculate_default_transform(
            src.crs, TARGET_CRS, src.width, src.height, *src.bounds
        )
        kwargs = src.meta.copy()
        kwargs.update({
            "crs": TARGET_CRS,
            "transform": transform,
            "width": width,
            "height": height
        })
        with rasterio.open(out_path, "w", **kwargs) as dst:
            for i in range(1, src.count + 1):
                reproject(
                    source=rasterio.band(src, i),
                    destination=rasterio.band(dst, i),
                    src_transform=src.transform,
                    src_crs=src.crs,
                    dst_transform=transform,
                    dst_crs=TARGET_CRS,
                    resampling=Resampling.nearest
                )
        return out_path

# Processar cada TIFF
for tiff_path in tiff_files:
    m = re.search(r"(\d{4})", tiff_path)
    year = int(m.group(1))

    # Escolher COS correto
    gdf = cos_2018 if year <= 2020 else cos_2023
    print(f"\n🟢 Processando {tiff_path} com COS {2018 if year <= 2020 else 2023}")

    # Reprojetar TIFF se necessário
    tmp_path = tiff_path.replace(".tif", "_utm29n_tmp.tif")
    tiff_proj = reproject_to_target(tiff_path, tmp_path)

    # Abrir raster reprojetado
    with rasterio.open(tiff_proj) as burned_src:
        burned = burned_src.read(1).astype(np.float32)
        meta = burned_src.meta.copy()
        nodata = burned_src.nodata
        burned[burned == nodata] = np.nan

        # Rasterizar máscara de vegetação diretamente, sem unir geometrias
        veg_mask = features.rasterize(
            [(geom, 1) for geom in gdf.geometry],
            out_shape=(burned_src.height, burned_src.width),
            transform=burned_src.transform,
            fill=0,
            dtype=np.uint8
        ).astype(bool)

        # Substituir NaN por 0 apenas dentro da vegetação
        nan_mask = np.isnan(burned)
        burned[nan_mask & veg_mask] = 0
        burned = np.nan_to_num(burned, nan=nodata if nodata is not None else 0)

        # Guardar TIFF final
        out_path = tiff_path.replace(".tif", "_p.tif")
        meta.update(dtype=rasterio.float32, crs=TARGET_CRS)
        with rasterio.open(out_path, "w", **meta) as dst:
            dst.write(burned, 1)

        print(f"✅ Guardado: {out_path}")

    # Apagar TIFF temporário
    if os.path.exists(tiff_proj) and tiff_proj.endswith("_utm29n_tmp.tif"):
        os.remove(tiff_proj)



📖 A carregar COS2018v3-S2.shp ...
✅ 350394 polígonos de vegetação filtrados.
🔄 Reprojetando COS → EPSG:32629

📖 A carregar COS2023v1-S2.shp ...
✅ 350449 polígonos de vegetação filtrados.
🔄 Reprojetando COS → EPSG:32629

🟢 Processando ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2015.tif com COS 2018
✅ Guardado: ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2015_p.tif

🟢 Processando ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2015_utm29n_tmp.tif com COS 2018
🔄 Reprojetando years_since_fire_2015_utm29n_tmp.tif → EPSG:32629
✅ Guardado: ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2015_utm29n_tmp_p.tif

🟢 Processando ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2016.tif com COS 2018
✅ Guardado: ..\..\Data\Interim\GIS_data\years_since_last_fire\PT-FireSprd\years_since_fire_2016_p.tif

🟢 Processando ..\..\Data\Interim\GIS_d